# COARR
음식의 맛은 20점 만점  
식당의 서비스, 분위기는 10점 만점

In [1]:
import os
import openai
import deepl
import json
import time

### API Keys

### Translate via DeepL

In [3]:
translator = deepl.Translator(deepl_api_key)

def ko2en(text_ko):
    return translator.translate_text(text_ko, source_lang="KO", target_lang="EN-US").text

def en2ko(text_en):
    return translator.translate_text(text_en, target_lang="KO").text

### COARR in English

In [4]:
def coarr_en(review_en):
    prompt1 = """\
    Evaluate the professionalism of the following restaurant review based on two criteria
    1. including detailed and illustrative description of the flavors, textures, ingredients, cooking techniques, and overall taste experience of the food (score out of 20)
    2. including detailed and illustrative description of the dining experience, service quality, the atmosphere of the restaurant, and the attentiveness of the staff (score out of 10)
    Review:
    """ + review_en

    message1=[{"role": "user", "content": prompt1}]
    response1 = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages = message1,
        temperature=0.2,
        max_tokens=2048,
        frequency_penalty=0.0
    )
    
    prompt2 = """\
    return the article in json format without the overall evaluation
    format:
    {
        "criterion1": {
            "ratings": "0/20",
            "evalutation": "..."
        },
        "criterion2": {
            "ratings": "0/10",
            "evaluation": "..."
        }
    }
    article:
    """ + response1.choices[0].message.content
    
    message2=[{"role": "user", "content": prompt2}]
    response2 = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages = message2,
        temperature=0.2,
        max_tokens=2048,
        frequency_penalty=0.0
    )
    
    return response2.choices[0].message.content

### Wrapping up

In [5]:
def coarr_ko(review_ko):
    start = time.time()
    
    review_en = ko2en(review_ko)
    eval_en = coarr_en(review_en)
    eval_json = json.loads(eval_en)
    
    # ratings
    eval_json["criterion1"]["ratings"] = int(eval_json["criterion1"]["ratings"][:-3])
    eval_json["criterion2"]["ratings"] = int(eval_json["criterion2"]["ratings"][:-3])
    
    # evaluation
    eval_json["criterion1"]["evaluation"] = en2ko(eval_json["criterion1"]["evaluation"])
    eval_json["criterion2"]["evaluation"] = en2ko(eval_json["criterion2"]["evaluation"])
    
    end = time.time()
    
    print(f"### Took {end - start:.2f} sec ###\n")
    print("### original evaluation in english ###")
    print(eval_en)
    
    return eval_json

### Examples

In [6]:
review_ko = """\
<20. 옥돔 구이> 옥돔 구이는 빵도 좋고 비늘도 살려서 잘 구웠는데 비늘이 억세서 먹기 조금 거슬렸다. 아쉬웠던 부분. <21. 전어> 좋아하는지 여쭤보시고는 두 점 주신 전어. 6일 숙성했다고 하는데 숙성 일수에 비해 멸치향 전혀 없고 육즙? 어즙?도 제법 있는, 호불호 덜 갈리는 맛있는 전어였다. + 셰프님이 다른 손님들께 전어를 주면서 전어에 대해 이런저런 얘기를 하는 중 다른 업장에서 근무할 때 숙성이 빡센 전어를 손님이 앞에서 뱉는 걸 보곤 그 이후로 강하게 숙성 안 한다는 일화를 들을 수 있었다. 궁금증. 분명 이 전어도 맛있는 축에 속하는 전어인데, 장성태 셰프 전어와의 차이점은 도대체 어디서 오는 걸까? 원물의 차이일까, 숙성에 쓰는 식초의 차이일까, 아니면 숙성일 차이일까. 정말 궁금하다. <22. 붕장어> 붕장어는 살맛도 소스도 둘 다 진해서 괜찮았는데 겉이 조금만 더 바삭했으면 훨씬 맛있었겠다 싶었다. <23. 앵콜 전어> 앵콜은 전어로. 다른 손님들도 대부분 전어를 요청했다. 얼마나 많이 드실 수 있냐고 여쭤보시길래 주시는 만큼 먹을 수 있다 했더니 세 점이나 주셨다.(다섯 점 주셨어도 충분히 먹을 수 있...) <24. 계란 구이> 스시렌의 또 다른 시그니처라 할 수 있는 계란 구이. 계란 구이가 맛있는 다른 어떤 업장보다 달고 맛있었다. 사람들이 왜 그렇게 좋아하는지 단번에 알 수 있었다. 다른 손님이 포장되는지 여쭤봐서 나도 반 판 포장을 할까 잠시 고민하다 하지는 않았다. 다음에 기회가 있다면 포장을 해봐야겠다."""

coarr_ko(review_ko)

### Took 22.17 sec ###

### original evaluation in english ###
{
    "criterion1": {
        "ratings": "12/20",
        "evaluation": "The review provides some description of the flavors, such as mentioning the abalone being delicious with juiciness and fish juice. However, it lacks specific details about the textures, ingredients, and cooking techniques used for each dish. The review also mentions the grilled tilefish being uncomfortable to eat due to strong scales, but it does not elaborate on the taste experience in detail. Overall, the review lacks comprehensive and detailed descriptions of the food, resulting in a lower score."
    },
    "criterion2": {
        "ratings": "6/10",
        "evaluation": "The review briefly mentions the dining experience, such as the chef asking about the dish and sharing an anecdote about over-aging abalone. However, it does not provide detailed descriptions of the service quality, atmosphere, or staff attentiveness. The review also mentions the a

{'criterion1': {'ratings': 12,
  'evaluation': '이 리뷰에서는 전복의 육즙과 생선즙이 맛있다고 언급하는 등 맛에 대한 설명이 일부 제공됩니다. 하지만 각 요리에 사용된 식감, 재료, 조리 기법에 대한 구체적인 설명이 부족합니다. 또한 옥돔 구이는 비늘이 강해 먹기 불편하다고 언급하고 있지만 맛에 대한 자세한 설명은 없습니다. 전반적으로 음식에 대한 포괄적이고 자세한 설명이 부족해 점수가 낮습니다.'},
 'criterion2': {'ratings': 6,
  'evaluation': '이 리뷰에는 셰프가 요리에 대해 묻고 전복 숙성에 대한 일화를 공유하는 등 식사 경험에 대해 간략하게 언급되어 있습니다. 그러나 서비스 품질, 분위기 또는 직원의 세심함에 대한 자세한 설명은 제공하지 않습니다. 이 리뷰에는 구운 달걀이 있는지 여부와 다음 번에는 구운 달걀을 먹어보고 싶다는 내용도 언급되어 있습니다. 식사 경험에 대한 통찰력을 제공하지만 깊이와 구체성이 부족합니다.'}}

In [7]:
review_ko = """\
<11. 흑점줄전갱이>스시 두 번째는 흑점줄전갱이. 숙성을 너무 오래 했는지 기대했던 서걱거림이 전혀 느껴지지 않고 그냥 녹아 없어지기만 했다. 아쉽. 안에는 시소와 실파를 갈아서 만든 페이스트가 들어가 있었다. 갑자기 안주하라며 김도 몇 장. <12. 새끼 황돔> 새끼 황돔은 위에 유자 제스트를 뿌려서 주셨는데 향긋하니 황돔의 가벼운 맛과 잘 어울렸다. 어디든 새끼 황돔은 위에 유자 제스트를 많이 뿌려서 나오는데 아마도 이런 이유에서인가 문득 생각이 들었다. <13. 단새우> 단새우는 안에 새우 내장으로 만든 소스 비슷한 뭔가를 넣어서 주셨는데, 단새우 맛 자체도 신선해서 아주 녹진했는데 내장 소스가 피니쉬가 아주 오래 남을 정도로 진한 맛이 있었다. 기대 안 했는데 의외로 괜찮았던 부분. <14. 참다랑어 속살> 이날 준비된 참다랑어는 아일랜드 자연산. 즈께는 오래 했는데 산미는 그리 인상적이지 않았다. 먹고나서 산지는 나중에 들었는데 듣고나니 조금 아쉬운?"""

coarr_ko(review_ko)

### Took 19.23 sec ###

### original evaluation in english ###
{
    "criterion1": {
        "ratings": "12/20",
        "evaluation": "The review provides some description of the flavors and ingredients of each dish, but lacks detailed and illustrative descriptions of the textures, cooking techniques, and overall taste experience. The reviewer's comments on the flavors are subjective and do not provide a comprehensive understanding of the dishes."
    },
    "criterion2": {
        "ratings": "4/10",
        "evaluation": "The review briefly mentions being given seaweed to settle down and expresses disappointment after hearing the origin of the bluefin tuna. However, there is no detailed description of the dining experience, service quality, or the atmosphere of the restaurant. The reviewer does not provide any information about the attentiveness of the staff or the overall ambiance."
    }
}


{'criterion1': {'ratings': 12,
  'evaluation': '이 리뷰에는 각 요리의 맛과 재료에 대한 설명이 일부 포함되어 있지만 식감, 요리 기술 및 전반적인 맛 경험에 대한 상세하고 예시적인 설명이 부족합니다. 맛에 대한 리뷰어의 의견은 주관적이며 요리에 대한 포괄적인 이해를 제공하지 않습니다.'},
 'criterion2': {'ratings': 4,
  'evaluation': '이 리뷰에서는 해초가 제공되었다고 간략하게 언급하고 참다랑어의 원산지에 대해 들은 후 실망감을 표현했습니다. 그러나 식사 경험, 서비스 품질 또는 레스토랑 분위기에 대한 자세한 설명은 없습니다. 이 리뷰어는 직원의 친절도나 전반적인 분위기에 대한 정보를 제공하지 않습니다.'}}